In [26]:
#import json lib to get nice and valid json 
import json
from datetime import datetime, timedelta
import spiceypy as spice

#check if spiceypy is imported
spice.tkvrsn("TOOLKIT") 

'CSPICE_N0066'

In [27]:
#load kernels
spice.furnsh("./cassMetaK.txt") 

In [28]:
#moon general info 
#https://solarsystem.nasa.gov/planets/IAPETUS/facts

#initial position is the end of the mission 
init_date = spice.str2et('2017-09-15T10:32:00') 

planets = [{"name": "saturn",
          "radius": 58232.0,
          "img": "img/saturn.jpg"
         },
         {"name": "mimas",
          "radius": 198.2,
          "orbital_period": 22.61,
          "img": "img/mimas-sm.jpg"
         },
         {"name": "enceladus",
          "radius": 252.1,
          "orbital_period": 32.88,
          "img": "img/enceladus-sm.jpg"
         },
         {"name": "tethys",
          "radius": 533.0,
          "orbital_period": 45.31,
          "img": "img/tethys-sm.jpg"
         },
         {"name": "dione",
          "radius": 561.7,
          "orbital_period": 65.69,
          "img": "img/dione-sm.jpg"
         },
         {"name": "rhea",
          "radius": 764.3,
          "orbital_period": 108.42,
          "img": "img/rhea-sm.jpg"
         },
          {"name": "titan",
          "radius": 2574.7,
          "orbital_period": 382.7,
          "img": "img/titan-sm-brown.jpg"
         },
         {"name": "iapetus",
          "radius": 735.6,
          "orbital_period": 1904.0,
          "img": "img/iapetus-sm.jpg"
         }
        ]

In [29]:
# global vars to calculate moon's orbit
step = 1000 #1000 posiitions of the planet during the orbital period 
end_date  = datetime.strptime('2017-09-15T10:32:00', "%Y-%m-%dT%H:%M:%S")

In [30]:
 moons_data = []

for i, planet in enumerate(planets):
    
    # get initial position 
    init_position, init_lightTime = spice.spkpos(planet["name"].upper(), init_date, 'J2000', 'NONE', 'SATURN')
    
    if planet["name"] == "saturn":
        planet["radius"] = planet["radius"]/1000
        planet["pos_x"] = 0
        planet["pos_y"] = 0
        planet["pos_z"] = 0
        planet["class"] = "moon"
    else:
        #get orbit coordinates 
        orbital_period = planet["orbital_period"]
        start_date = end_date - timedelta(hours = orbital_period) #end date of orbital period
        utc = [start_date.strftime("%Y-%m-%dT%H:%M:%S"), end_date.strftime("%Y-%m-%dT%H:%M:%S")]

        etOne = spice.str2et(utc[0])
        etTwo = spice.str2et(utc[1])

        times = [x*(etTwo-etOne)/step + etOne for x in range(step+1)]
        times_utc = map(lambda t : spice.timout(t, "YYYY-MM-DD HR:MN:SC", 20), times)


        positions, lightTimes = spice.spkpos(planet["name"].upper(), times, 'J2000', 'NONE', 'SATURN')
        orbit_positions = []
        
        # position on orbit
        for old_dict in positions:
            new_dict = {"x": round(old_dict[1]/1000, 5), 
                        "y": round(old_dict[2]/1000, 5),
                        "z": round(old_dict[0]/1000, 5)}
            orbit_positions.append(new_dict)

        # initial position and radius in 1000
        planet["pos_x"] = round(init_position[1]/1000, 5),
        planet["pos_y"] = round(init_position[2]/1000, 5),
        planet["pos_z"] = round(init_position[0]/1000, 5),
        planet["radius"] = planet["radius"]/1000
        planet["orbit"] = orbit_positions
        planet["class"] = "moon"

    
        
# print Mimas data        
print(planets[0])

{'pos_z': 0, 'pos_x': 0, 'pos_y': 0, 'name': 'saturn', 'img': 'img/saturn.jpg', 'radius': 58.232, 'class': 'moon'}


In [32]:
# write to json
with open("planets.json", "w") as f:
    f.write("%s" % json.dumps(planets)) #write json